In [ ]:
# This notebook is based largely on @mf1024's "Teach GPT-2 a sense of humour"
# I've graciously borrowed much of his logic for fine tuning and generating from his implmenetation
# Check out his notebook here https://github.com/mf1024/Transformers/blob/master/Teaching%20GPT-2%20a%20sense%20of%20humor.ipynb

In [94]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import numpy as np
import os

In [2]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

In [4]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
model = GPT2LMHeadModel.from_pretrained('gpt2-medium')
model = model.to(device)


100%|██████████| 1520013706/1520013706 [01:46<00:00, 14323311.21B/s]


In [90]:
def choose_top_n(probs, n=5):
    ind = np.argpartition(probs,-n)[-n:]
    top_probs = probs[ind]
    print(top_probs)
    top_probs = top_probs / np.sum(top_probs)
    choice = np.random.choice(n,1,p= top_probs)
    token_id = ind[choice][0]
    
    return int(token_id)

In [91]:
choose_top_n(np.array([0.99,0.6,0.6,0.91,0.9,0.6,0.88]))

[0.6  0.91 0.9  0.99 0.88]


0

In [102]:
FILE_PATH = os.path.join("data", "film_text.txt")

In [132]:
class ScriptDataset(Dataset):
    def __init__(self):
        super().__init__()
        assert os.path.isfile(FILE_PATH)
        directory, filename = os.path.split(FILE_PATH)
        
        with open(FILE_PATH, "rb") as f:
            data = f.readlines()
            
        self.script_text  = []
        self.end_of_text_token = "<|endoftext|>"
        for i, script_entry in enumerate(data):
            script_str = f"{self.end_of_text_token}START:{script_entry}{self.end_of_text_token}"
            self.script_text.append(script_str)
        
        

In [133]:
sd = ScriptDataset()

["<|endoftext|>START:b'\\n'<|endoftext|>", "<|endoftext|>START:b'                 STACEY\\n'<|endoftext|>", "<|endoftext|>START:b'Fucking pervert, Dan!\\n'<|endoftext|>", "<|endoftext|>START:b'\\n'<|endoftext|>", "<|endoftext|>START:b'\\n'<|endoftext|>", "<|endoftext|>START:b'          DANNY\\n'<|endoftext|>", '<|endoftext|>START:b"There\'s a black guy outside Der...\\n"<|endoftext|>', "<|endoftext|>START:b'breaking into your car.\\n'<|endoftext|>", "<|endoftext|>START:b'\\n'<|endoftext|>", "<|endoftext|>START:b'Derek, muscled and tattooed, jumps out of the bed and\\n'<|endoftext|>", "<|endoftext|>START:b'quickly puts on his skivvies. He reaches under his\\n'<|endoftext|>", "<|endoftext|>START:b'mattress, pulls out a SIG .45 semi-automatic pistol, and\\n'<|endoftext|>", "<|endoftext|>START:b'shoves in a clip.\\n'<|endoftext|>", "<|endoftext|>START:b'\\n'<|endoftext|>", "<|endoftext|>START:b'\\n'<|endoftext|>", "<|endoftext|>START:b'              DEREK\\n'<|endoftext|>", "<|endoftext|>S